In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# !cp  -r '/content/drive/MyDrive/Hackathon_HiGency/data/images_stanford' '/content'

## Librairies

In [2]:
import pandas as pd
import numpy as np
from PIL import Image
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from tqdm import tqdm
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input,decode_predictions
import numpy as np
from keras.applications.vgg16 import VGG16
from sklearn.model_selection import train_test_split
import tensorflow as tf
import numpy as np
from keras.applications.vgg16 import VGG16
from sklearn.model_selection import train_test_split
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import densenet
from keras.models import Sequential, Model, load_model
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, Callback
from keras import regularizers
from keras import backend as K
from tensorflow.keras.optimizers import Adam

training = False

## Data

In [3]:
img_width, img_height = 224, 224
nb_train_samples = 6616
nb_validation_samples = 1654
epochs = 10
batch_size = 32
n_classes = 100

In [4]:
if training:
  data_stanford_df = pd.read_pickle("/content/drive/MyDrive/Hackathon_HiGency/data/data_stanford.pkl")
  data_stanford_df

,im_name,x_min,y_min,x_max,y_max,class,models,in_original,width,height
0,000001.jpg,112,7,853,717,car,AM General Hummer SUV 2000,True,1024.0,768.0
1,000002.jpg,48,24,441,202,car,AM General Hummer SUV 2000,True,460.0,219.0
2,000003.jpg,7,4,277,180,car,AM General Hummer SUV 2000,True,310.0,233.0
3,000004.jpg,33,50,197,150,car,AM General Hummer SUV 2000,True,250.0,188.0
4,000005.jpg,5,8,83,58,car,AM General Hummer SUV 2000,True,85.0,64.0
...,...,...,...,...,...,...,...,...,...,...
16180,016181.jpg,38,36,375,234,car,smart fortwo Convertible 2012,False,416.0,244.0
16181,016182.jpg,29,34,235,164,car,smart fortwo Convertible 2012,False,259.0,194.0
16182,016183.jpg,25,32,587,359,car,smart fortwo Convertible 2012,False,600.0,432.0
16183,016184.jpg,56,60,208,186,car,smart fortwo Convertible 2012,False,259.0,194.0


In [5]:
if training:
  df_train = data_stanford_df[data_stanford_df['in_original']==True]
  df_train = df_train.drop(columns=['x_min',	'y_min',	'x_max',	'y_max',	'class', 'in_original','width',	'height'])
  df_train =df_train.rename(columns={"im_name": "filename"})
  train_df, test_df = train_test_split(df_train, test_size=0.2)

In [6]:
# Load the the carbon models
df_carbon = pd.read_csv('/content/drive/MyDrive/Hackathon_HiGency/data/car_models_footprint.csv', sep=';')
df_carbon

,models,Brand,year,Average Urban Consumption,Average extra-urban consumption,Average mixed consumption,Average of CO2 (g per km),Average CO type I
0,Ford Fiesta Sedan 2012,Ford,2012,4.900000,3.600,4.100000,107.000000,"0,421"
1,FIAT 500 Convertible 2012,FIAT,2012,5.725000,4.175,4.775000,111.000000,"0,318"
2,Chevrolet Corvette ZR1 2012,Chevrolet,2012,19.000000,8.900,12.600000,293.000000,"0,508"
3,Lamborghini Gallardo LP 570-4 Superleggera 2012,Lamborghini,2012,21.406667,9.860,14.086667,337.133333,0
4,Honda Accord Coupe 2012,Honda,2012,6.900000,4.500,5.400000,141.000000,"0,215"
...,...,...,...,...,...,...,...,...
95,Rolls-Royce Phantom Sedan 2012,Rolls-Royce,2012,NaN,NaN,NaN,113.000000,NaN
96,Rolls-Royce Phantom Drophead Coupe Convertible...,Rolls-Royce,2012,NaN,NaN,NaN,121.000000,NaN
97,Scion xD Hatchback 2012,Scion,2012,NaN,NaN,NaN,208.000000,NaN
98,Spyker C8 Coupe 2009,Spyker,2009,NaN,NaN,NaN,131.000000,NaN


In [7]:
if training:
  train_df['class'] = train_df['models'].apply(lambda x : df_carbon[df_carbon['models']==x]['Average of CO2 (g per km)'].values[0])
  test_df['class'] = test_df['models'].apply(lambda x : df_carbon[df_carbon['models']==x]['Average of CO2 (g per km)'].values[0])
  train_df

In [10]:
if training:
  %cd '/content/drive/MyDrive/Hackathon_HiGency/data/images_stanford'
  train_datagen = ImageDataGenerator(featurewise_center=True,
                                     rotation_range=90,
                                     horizontal_flip=True,
                                     vertical_flip=True,
                                     zoom_range=0.2
                                     )
  #train_datagen = ImageDataGenerator()
  
  
  test_datagen = ImageDataGenerator(featurewise_center=True,
                                     rotation_range=90,
                                     horizontal_flip=True,
                                     vertical_flip=True,
                                     zoom_range=0.2)
  #test_datagen = ImageDataGenerator()
  
  train_datagen.mean = [123.68, 116.779, 103.939]
  test_datagen.mean = [123.68, 116.779, 103.939]

  train_generator = train_datagen.flow_from_dataframe(
      train_df,
      target_size=(img_width, img_height),
      batch_size=batch_size,
      class_mode="raw")
  
  validation_generator = test_datagen.flow_from_dataframe(
      test_df,
      target_size=(img_width, img_height),
      batch_size=batch_size,
      class_mode="raw")

/content/drive/.shortcut-targets-by-id/1TwodLZHcjY5V5pFsayKXQpZZAoZTRYeA/Hackathon_HiGency/data/images_stanford


## Pretrained

In [12]:
from tensorflow.keras.applications import EfficientNetB0

In [13]:
effnet_model = Sequential()

pretrained_model= EfficientNetB0(include_top=False,
                   input_shape=(224,224,3),
                   pooling='avg',
                   weights='imagenet')

for layer in pretrained_model.layers:
        layer.trainable=False

effnet_model.add(pretrained_model)

In [14]:
effnet_model.add(Flatten())
effnet_model.add(Dense(512, activation='relu'))
effnet_model.add(Dense(1, activation='linear'))

In [15]:
if training:
  effnet_model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_absolute_percentage_error', metrics=[tf.keras.metrics.MeanAbsolutePercentageError()])
  early_stop = EarlyStopping(monitor='val_loss', patience=8, verbose=1, min_delta=1e-4)
  reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=4, verbose=1, min_delta=1e-4)
  checkpoint = ModelCheckpoint('effnet_finetune.h15', monitor= 'val_loss', save_best_only = True, verbose= 1)
  callbacks_list = [early_stop, reduce_lr,checkpoint]
  
  history = effnet_model.fit(train_generator, validation_data=validation_generator, epochs=100, callbacks=callbacks_list)

Epoch 1/100
207/207 [==============================] - ETA: 0s - loss: 33.6175 - mean_absolute_percentage_error: 33.6175
Epoch 1: val_loss improved from inf to 25.84919, saving model to effnet_finetune.h15
INFO:tensorflow:Assets written to: effnet_finetune.h15/assets
207/207 [==============================] - 240s 1s/step - loss: 33.6175 - mean_absolute_percentage_error: 33.6175 - val_loss: 25.8492 - val_mean_absolute_percentage_error: 25.8492 - lr: 0.0010
Epoch 2/100
207/207 [==============================] - ETA: 0s - loss: 25.6397 - mean_absolute_percentage_error: 25.6397
Epoch 2: val_loss improved from 25.84919 to 25.12102, saving model to effnet_finetune.h15
INFO:tensorflow:Assets written to: effnet_finetune.h15/assets
207/207 [==============================] - 222s 1s/step - loss: 25.6397 - mean_absolute_percentage_error: 25.6397 - val_loss: 25.1210 - val_mean_absolute_percentage_error: 25.1210 - lr: 0.0010
Epoch 3/100
207/207 [==============================] - ETA: 0s - loss: 25

KeyboardInterrupt: ignored

### Results

In [ ]:
y_true = test_df['class_co2'].values
print(y_true.shape)
print(np.mean((np.abs(y_true-y_pred[:,0])/y_true))) #0.2776463470246597 on images_hackathon

In [ ]:
if training:
  tf.keras.models.save_model(effnet_model, '/content/drive/MyDrive/Hackathon_HiGency/Step2/effnet_reg')
else :
  effnet_model.load('/content/drive/MyDrive/Hackathon_HiGency/Step2/effnet_reg')

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Hackathon_HiGency/Step2/effnet_reg/assets


(None,)

##  Final prediction

In [64]:
cd /content/drive/MyDrive/Hackathon_HiGency/data

/content/drive/.shortcut-targets-by-id/1TwodLZHcjY5V5pFsayKXQpZZAoZTRYeA/Hackathon_HiGency/data


In [65]:
#from Walid (step 1)
train_annotation = pd.read_pickle('test_car_annotations_fixed.pkl')
train_annotation

,im_name,x_min,y_min,x_max,y_max,class
3,71540198.jpg,98,79,1620,815,1
4,85427036.jpg,234,195,1033,643,1
6,6087324064.jpg,1480,371,1602,705,1
8,11828073.jpg,19,62,614,417,1
11,5116251.jpg,96,77,1081,559,1
...,...,...,...,...,...,...
1015,38559539.jpg,116,90,841,447,1
1017,77082581.jpg,115,323,780,555,1
1018,89862599.jpg,23,109,970,571,1
1021,73537501.jpg,84,81,525,376,1


In [62]:
train_annotation[['x_min', 'y_min', 'y_max', 'x_max']].values.sum()

822054

In [51]:
test_df = train_annotation[train_annotation['class']==1]
test_df

In [54]:
cd /content/drive/MyDrive/Hackathon_HiGency/data/dataset_set

/content/drive/.shortcut-targets-by-id/1TwodLZHcjY5V5pFsayKXQpZZAoZTRYeA/Hackathon_HiGency/data/dataset_set


In [55]:
test_final_datagen = ImageDataGenerator(featurewise_center=True)

test_final_datagen.mean = [123.68, 116.779, 103.939]

test_generator = train_datagen.flow_from_dataframe(
    test_df,
    x_col='im_name', 
    target_size=(img_width, img_height),
    batch_size=batch_size,
    shuffle=False,
    class_mode=None)

Found 532 validated image filenames.


In [61]:
%%time
y_pred = effnet_model.predict(test_generator)

CPU times: user 15.9 s, sys: 293 ms, total: 16.2 s
Wall time: 15.9 s


In [67]:
np.sum(y_pred)

86819.7

In [66]:
#to Louis 
np.save('pred_carbon.npy', y_pred)
test_df.to_pickle('pred_carbon_df.pkl')